## Energy Resolution of Germanium detectors

Eugenia Boccanera 2109310  
Lucrezia Rossi ...

Germanium detectors have wide fields of application for $\gamma-$ and $X-$ ray spectonometry thanks to their excellent energy resolution. The energy resolution of these detectors is defined as the width of the detected energy spectra peaks (FWHM); it depends on:
    - the statistics of the charge creation process
    - the properties of the detector, and primarly its charge cllelction efficency - the electronics noise

The resolution can be expressed as the suquared sum of two terms:   
$$FWHM = \sqrt{(w_d)^2 + (w_e)^2}$$   
where the first term depends on teh detector properties as:  
$w_d = 2 * \sqrt{(2 * \log(2)) * F * E_{\gamma} * w}$   
with $F$ the _Fano factor_,  
$E_{\gamma}$  the energy of the photon deposited energy(?)  
$w$ the electron hole-porduced energy threshold in germanium (w ~ 3eV).

The $w_e$ term in eq.1 is connected with the readout electronics and depends on the detector capacitance, the size of the detector and the bias voltage.

Detailed Steps
##### 1. Understand the Data and Model

The energy resolution (FWHM) of the Germanium detectors is given by:  
$$FWHM = \sqrt{(w_d)^2 + (w_e)^2}$$


with  
$w_d = 2 * \sqrt{(2 * \log(2)) * F * E_{\gamma} * w}$  


    ​

    Here, F is the Fano factor, $E_{\gamma}$​ is the energy of the photon, and w is the electron-hole production energy threshold in Germanium (approximately 3 eV).

    You need to determine w_d_​ and w_e_​ for your data.

##### 2. Define the Model for Fitting Peaks

Identify the function you will use to fit the peaks. A common choice for spectral peaks is the Gaussian function.
Define the full model incorporating the equation for FWHM and how it relates to the observed data.

Use a Gaussian function to model the peaks in the energy spectra:
$$f(E)=A * exp⁡(−\frac{(E−μ)^2}{2σ^2})$$
    
where μ is the peak center, σ is the standard deviation related to FWHM by $FWHM = 2 * \sqrt{2 * \log{2}} * \sigma$, and A is the amplitude.

##### 3. Set Up Bayesian Inference

Define prior distributions for the parameters μ, σ, A, $w_d$​, and $w_e$​. For example:  
$μ∼N(μ_0,(σ_{\mu})^2)$
$σ∼N(σ_0,σ_σ^2)$  
$A∼N(A_0,σ_A^2)$  
$w_d∼N(w_{d0},σ_{w_d}^2)$  
$w_e∼N(w_{e0},σ_{w_e}^2)$

where N denotes a normal distribution with specified mean and variance.

##### 4. Perform Bayesian Inference

Use a Bayesian inference library like Stan to perform Markov Chain Monte Carlo (MCMC) sampling to obtain the posterior distributions of the parameters.



##### 5. Validate and Interpret the Results

Check the trace plots and posterior distributions to ensure the MCMC chains have converged.
Compare the fitted model to the observed data using posterior predictive checks.

##### 6. Report the Results

Summarize the parameter estimates, including credible intervals.
Provide visualizations, such as trace plots and the fitted spectrum overlaid on the observed data.

### Theory Behind Bayesian Inference

Bayesian inference combines prior knowledge with observed data to update the probability of a hypothesis. The steps are:

- Prior Distribution: Represents your initial beliefs about the parameters before seeing the data.   
- Likelihood: Represents the probability of the observed data given the parameters.  
- Posterior Distribution: Updated beliefs about the parameters after considering the data, obtained via Bayes' theorem:  
    $P(θ∣data) ∝ P(data∣θ)P(θ)$


MCMC methods, such as the Metropolis-Hastings algorithm, are used to sample from the posterior distribution when it cannot be computed analytically.

Metropolis-Hastings: Proposes new parameter values and accepts them based on an acceptance criterion.


### 1. Load the required libraries

In [ ]:
# Load the libraries
library(rstan)
library(ggplot2)
library(bayesplot)

# Configure rstan
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())


Caricamento del pacchetto richiesto: StanHeaders


rstan version 2.32.6 (Stan version 2.32.2)


For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
For within-chain threading using `reduce_sum()` or `map_rect()` Stan functions,
change `threads_per_chain` option:
rstan_options(threads_per_chain = 1)


Do not specify '-march=native' in 'LOCAL_CPPFLAGS' or a Makevars file

This is bayesplot version 1.11.1

- Online documentation and vignettes at mc-stan.org/bayesplot

- bayesplot theme set to bayesplot::theme_default()

   * Does _not_ affect other ggplot2 plots

   * See ?bayesplot_theme_set for details on theme setting



### 2. Define the Stan Model
Create a Stan model file 'peak_fitting.stan' with this content:

data {
  int<lower=0> N;  // Number of data points
  vector[N] E;     // Energy values
  vector[N] y;     // Observed counts
  real mu_prior;   // Prior mean for peak center
  real<lower=0> sigma_mu; // Prior std for peak center
  real<lower=0> sigma_prior; // Prior std for peak width
  real<lower=0> sigma_sigma; // Prior std for sigma
  real A_prior;    // Prior mean for amplitude
  real<lower=0> sigma_A;     // Prior std for amplitude
}

parameters {
  real mu;         // Peak center
  real<lower=0> sigma; // Peak width
  real<lower=0> A;     // Peak amplitude
}

model {
  // Priors
  mu ~ normal(mu_prior, sigma_mu);
  sigma ~ normal(sigma_prior, sigma_sigma);
  A ~ normal(A_prior, sigma_A);
  
  // Likelihood
  y ~ normal(A * exp(-0.5 * square((E - mu) / sigma)), sigma);
}


### 3. Prepare data in R

In [2]:
# Sample data
E <- c(...)  # Energy values
y <- c(...)  # Observed counts

# Prior values
mu_prior <- 1000  # Example value
sigma_mu <- 10
sigma_prior <- 10
sigma_sigma <- 5
A_prior <- 100
sigma_A <- 10

# Data list for Stan
data_list <- list(
  N = length(E),
  E = E,
  y = y,
  mu_prior = mu_prior,
  sigma_mu = sigma_mu,
  sigma_prior = sigma_prior,
  sigma_sigma = sigma_sigma,
  A_prior = A_prior,
  sigma_A = sigma_A
)


ERROR: Error in eval(expr, envir, enclos): '...' utilizzato in un contesto sbagliato


### 4. Run Bayesian Inference 

In [ ]:
# Compile the model
stan_model <- stan_model("peak_fitting.stan")

# Fit the model
fit <- sampling(stan_model, data = data_list, iter = 2000, warmup = 1000, chains = 4, seed = 123)

# Print summary of the fit
print(fit)


### 5. Analyze and Interpre Results

In [ ]:
# Extract posterior samples
posterior <- extract(fit)

# Summary statistics
summary(fit)

# Trace plots to check convergence
traceplot(fit)


### 6. Visualize the Fit

In [ ]:
# Posterior predictive checks
y_rep <- posterior_predict(fit)

# Plot the observed data and the fit
ggplot() +
  geom_point(aes(x = E, y = y), color = 'blue') +
  geom_line(aes(x = E, y = apply(y_rep, 2, mean)), color = 'red') +
  labs(x = 'Energy (E)', y = 'Counts') +
  ggtitle('Observed Data and Fitted Model')